In [8]:
!pip3 install ultralytics
!pip3 install lapx>=0.5.2
!pip3 install --upgrade ultralytics



zsh:1: 0.5.2 not found
^C
ERROR: Operation cancelled by user


In [10]:
from ultralytics import YOLO
import cv2

In [ ]:
model = YOLO('/Users/shravanp/Coding/Robotics/AIScouter/src/models/y11v9.pt')

In [17]:
video_source = '/Users/shravanp/Coding/Robotics/AIScouter/src/videos/dcmp58.mp4'
output_path = '/Users/shravanp/Coding/Robotics/AIScouter/src/videos/output/dcmp58-bytetrack.mp4'

# Capture the video source to get frame properties
video_capture = cv2.VideoCapture(video_source)
fps = video_capture.get(cv2.CAP_PROP_FPS)
frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Set up the video writer to save the output video
video_writer = cv2.VideoWriter(
    output_path, 
    cv2.VideoWriter_fourcc(*'mp4v'), 
    fps, 
    (frame_width, frame_height)
)

# Run the tracking process
results = model.track(
    source=video_source,
    save=False,  # Avoid auto-saving
    tracker='/Users/shravanp/Coding/Robotics/AIScouter/src/config/bytetrack.yaml',
    persist=True
)

# Process each frame and save it to the output video
for result in results:
    frame = result.orig_img  # Original frame from the video
    for bbox in result.boxes:
        # Extract bounding box coordinates and class label
        x1, y1, x2, y2 = map(int, bbox.xyxy[0])  # Bounding box coordinates
        class_label = result.names[int(bbox.cls[0])]  # Get class label
        robot_id = str([int(bbox.id[0])])  # Get class label
          # Get tracked ID

        # Determine color based on the class label
        if class_label == 'Blue':
            color = (255, 0, 0)  # Blue color in BGR format
        elif class_label == 'Red':
            color = (0, 0, 255)  # Red color in BGR format
        else:
            color = (0, 255, 0)  # Default color (green) for other classes

        # Draw the bounding box with the specified color
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        
        # Optionally, add the label text
        cv2.putText(frame, f"{class_label} - {robot_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Write the modified frame to the output video
    video_writer.write(frame)

# Release resources
video_capture.release()
video_writer.release()
cv2.destroyAllWindows()

print(f"Processed video saved to {output_path}")



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/721) /Users/shravanp/Coding/Robotics/AIScouter/src/videos/dcmp58.mp4: 192x640 3 Blues, 2 Reds, 145.4ms
video 1/1 (frame 2/721) /Users/shravanp/Coding/Robotics/AIScouter/src/videos/dcmp58.mp4: 192x640 3 Blues, 2 Reds, 133.2ms
video 1/1 (frame 3/721) /Users/shravanp/Coding/Robotics/AIScouter/src/videos/dcmp58.mp4: 192x640 3 Blues, 3 Reds, 81.7ms
video 1/1 (frame 4/721) /Users/shravanp/Coding/Robotics/AIScouter/src/videos/dcmp58.mp4: 19

In [ ]:
!ffmpeg -i /notebooks/runs/detect/track5/dcmp58.avi -c:v libx264 -crf 23 -preset medium -c:a aac -b:a 128k rb.mp4 -y